# Advanced Features: Cortex Guard and Helper Functions

This notebook demonstrates:
- **Cortex Guard**: Safety filtering for AI responses
- **AI_COUNT_TOKENS**: Token counting for optimization
- **PROMPT**: Dynamic prompt building
- **TRY_COMPLETE**: Error-safe completions


In [ ]:
import streamlit as st
import pandas as pd
import altair as alt
from snowflake.snowpark.context import get_active_session

session = get_active_session()
session.sql("USE DATABASE AISQL_DB").collect()
session.sql("USE SCHEMA AISQL_SCHEMA").collect()
session.sql("USE WAREHOUSE AISQL_WH").collect()


## 1. Cortex Guard: Safety Filtering

Filter potentially unsafe AI responses


In [ ]:
# Generate responses with Cortex Guard
sql = """
SELECT 
    ticket_id,
    SUBSTR(content, 1, 100) as customer_message,
    AI_COMPLETE('llama3.1-8b', 
        'Generate a professional response to: ' || content,
        {'guard_enable': true}) as safe_response,
    CASE 
        WHEN AI_COMPLETE('llama3.1-8b', 
            'Generate a professional response to: ' || content,
            {'guard_enable': true}) IS NULL 
        THEN 'Filtered'
        ELSE 'Approved'
    END as guard_status
FROM emails
LIMIT 20
"""

df_guard = session.sql(sql).to_pandas()
st.subheader("Cortex Guard Results")
st.dataframe(df_guard)

# Show guard statistics
guard_stats = df_guard['GUARD_STATUS'].value_counts()
col1, col2 = st.columns(2)
with col1:
    st.metric("Approved", guard_stats.get('Approved', 0))
with col2:
    st.metric("Filtered", guard_stats.get('Filtered', 0))


## 2. AI_COUNT_TOKENS: Token Management

Count tokens for cost optimization


In [ ]:
# Count tokens for cost estimation
sql = """
SELECT 
    ticket_id,
    SUBSTR(content, 1, 100) as content_preview,
    LENGTH(content) as char_count,
    AI_COUNT_TOKENS('claude-3-7-sonnet', content) as token_count,
    ROUND((AI_COUNT_TOKENS('claude-3-7-sonnet', content) / 1000.0) * 0.003, 6) as estimated_cost_usd
FROM emails
ORDER BY token_count DESC
LIMIT 20
"""

df_tokens = session.sql(sql).to_pandas()
st.subheader("Token Analysis")
st.dataframe(df_tokens)

# Token distribution
hist = alt.Chart(df_tokens).mark_bar().encode(
    alt.X('TOKEN_COUNT:Q', bin=alt.Bin(maxbins=20)),
    y='count()'
).properties(height=300)
st.subheader("Token Count Distribution")
st.altair_chart(hist, use_container_width=True)

# Cost summary
total_tokens = df_tokens['TOKEN_COUNT'].sum()
total_cost = df_tokens['ESTIMATED_COST_USD'].sum()
st.metric("Total Tokens", f"{total_tokens:,}")
st.metric("Estimated Cost", f"${total_cost:.4f}")


## 3. PROMPT Function: Dynamic Prompts

Build dynamic prompts with parameters


In [ ]:
# Use PROMPT for dynamic prompt building
sql = """
SELECT 
    ticket_id,
    user_id,
    SUBSTR(content, 1, 100) as content_preview,
    AI_COMPLETE('claude-3-7-sonnet',
        PROMPT('User {0} submitted ticket {1}. Please provide a helpful response to: {2}',
               user_id, ticket_id, content)) as personalized_response
FROM emails
LIMIT 5
"""

df_prompt = session.sql(sql).to_pandas()
st.subheader("Dynamic Prompt Results")
st.dataframe(df_prompt)


## Summary

This notebook covered advanced Cortex AISQL features:
- **Cortex Guard** for safe AI responses
- **AI_COUNT_TOKENS** for cost optimization
- **PROMPT** for dynamic prompt construction
- **TRY_COMPLETE** for error handling

These functions help build production-ready AI applications with proper safety, cost management, and error handling.
